# FastAI Chapter 5 - summary

In chapter 5 the book expands the model built in Ch4 (3s and 7s) to more than 2 categories. In this example the task is to classify 37 different dog breeds, hence for a loss function we will need an activation for each category with the following properties: <br> 
- values btw 0 and 1 <br>
- differentiable <br>
- each category sums up to 1 <br>
<br>

The solution to this is the cross entropy loss function. 
This chapter also uses regex in order to identify filenames to label the whole image dataset.

So let's start with label the dog breed dataset:

In [ ]:
#!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

from fastbook import *
from fastai.vision.all import *

path = untar_data(URLs.PETS)
Path.BASE_PATH = path

Since the 'annotation' folder which is included in the pets dataset comprises locations rather than breed we will extract the labelling information from the file names:

In [ ]:
re.findall(r'(.+)_\d+.jpg$', fname.name)

We will use this as get_y labelling method in the DataBlock class using the RegesLabelling class. We are also using the block API. Reminder: DataBlock helps to then quickly create Datasets and DataLoaders.

In [ ]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

help(DataBlock)

Since Datablock is the foundation of everything to come we have to make sure it does what we want it to do. So we need to check and debugg. First we will take a look at one batch:

In [ ]:
dls.show_batch(nrows=1, ncols=3)

We can debug the DataBlock using the 'summary' method. In the following example we forgot to include the Resize function in the Datablock which leads to images with different size, hence an error when collating the tensor.

In [ ]:
#hide_output
pets1 = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'))
pets1.summary(path/"images")

As ground truth we should train a simple model and check the results:

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

Note! If we don't specifiy a loss function explicitly fastAI will try to chose an apropriate one. In case of images this is usually 'cross-entropy'.

# Cross-Entropy

Cross-Entropy is a combination of 'soft-max' and log-likelihood. softmax is scaling the activation between 0 and 1 and ensures that the sum of the activations of all categories adds up to 1. 
The log is going to emphasise small changes close to the optimum (e.g.: to get from 0.99 accuracy to 0.999). <br>
The corresponing class is:

In [ ]:
nn.CrossEntropyLoss(reduction='none')(acts, targ)

# Model interpretation
In order to interpret the result of the model we can use a confusion matrix. This can get confusing is cases of a lot of categories. But we can look at the most confused categories using: 

In [ ]:
interp.most_confused(min_val=5)

# Learn rate finder
In order to find the right 'lr' we use the following function:

In [ ]:
learn = cnn_learner(dls, resnet34, metrics=error_rate)
lr_min,lr_steep = learn.lr_find()

This will increases the learn rate step by step and computes the loss. Once the loss increases the learn rate is too high.  

# Freezing and tranfer learning
When we fine-tune a network we throw out the last or last few layers, put on new ones with randomly initialized weights, freeze the parameters of the original layers while training the new layers for a few epochs and then unfreeze all parameters for the following epochs of training. 

# Discriminative Learning

Different learning rate for different layers in case of transfer learning. Pretrained layers will not need a learning rate as high as the new layers. 